In [1]:
library(raster)
library(terra)


Loading required package: sp



terra 1.7.78



In [ ]:
f <- "/data-stonlcd_agg.grd"
r <- raster(f)

library(terra)
x <- rast(r)

# get the attributes
lev <- levels(r)[[1]]  
lev <- lev[, c("ID", "Land.Cover.Class")]
lev[,2] <- as.character(lev[,2])

x <- rast(r)
levels(x) <- lev
is.factor(x)